In [1]:
from __future__ import print_function
import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D
from keras.layers import LSTM, Activation 
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [14]:
import os
start_token = " "

with open("names.txt") as f:
    names = f.read()[:-1].split('\n')
    names = [start_token+name for name in names]

text = '\n'.join(names)

chars = sorted(list(set(text)))
print('total chars: {}'.format(len(chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 56


In [15]:
maxlen = 10
step = 3

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
print('First 10 sequences and next chars:')
for i in range(10):
    print('[{}]:[{}]'.format(sentences[i], next_chars[i]))

Number of sequences: 21268
First 10 sequences and next chars:
[ Abagael
 ]:[A]
[agael
 Aba]:[g]
[el
 Abagai]:[l]
[ Abagail
 ]:[A]
[agail
 Abb]:[e]
[il
 Abbe
 ]:[A]
[ Abbe
 Abb]:[e]
[be
 Abbey
]:[ ]
[ Abbey
 Ab]:[b]
[bey
 Abbi
]:[ ]


In [16]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [17]:
nb_units = 256

model = Sequential()

# Recurrent layers supported: SimpleRNN, LSTM, GRU:
model.add(LSTM(nb_units, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))

# To stack multiple RNN layers, all RNN layers except the last one need
# to have "return_sequences=True".  An example of using two RNN layers:
#model.add(SimpleRNN(16,
#                    input_shape=(maxlen, len(chars)),
#                    return_sequences=True))
#model.add(SimpleRNN(32))

model.add(Dense(units=len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               320512    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 56)                14392     
_________________________________________________________________
activation_3 (Activation)    (None, 56)                0         
Total params: 334,904
Trainable params: 334,904
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
sys.stdout = open('generated.txt', 'w')

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

class SampleResult(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):

        start_index = random.randint(0, len(text) - maxlen - 1)

        for diversity in [0.2, 0.5, 1.0, 1.2]:
            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence

            sys.stdout.write(generated)
                        
             
            

            for i in range(100):
                X = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    X[0, t, char_indices[char]] = 1.

                preds = self.model.predict(X, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char
                

                sys.stdout.write(next_char)
                sys.stdout.flush()
                
                
        
sample_callback = SampleResult()
model.fit(X, y,
          batch_size=128,
          epochs=10,
          callbacks=[sample_callback], verbose=0)

/Users/bedabratachoudhury/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


array(['eorgena\n G', 'eorgena\n Ge', 'eorgena\n Geo', ...,
       'Chastity\n Chaylie\n Chaylte\n Chayntee\n Chanette\n Ciake\n Cibemarie\n Tmemike\n Temia\n Temma\n Temmie\n Temmy\n Tena',
       'Chastity\n Chaylie\n Chaylte\n Chayntee\n Chanette\n Ciake\n Cibemarie\n Tmemike\n Temia\n Temma\n Temmie\n Temmy\n Tena\n',
       'Chastity\n Chaylie\n Chaylte\n Chayntee\n Chanette\n Ciake\n Cibemarie\n Tmemike\n Temia\n Temma\n Temmie\n Temmy\n Tena\n '],
      dtype='|S110')

In [43]:
out = np.array(out)

In [44]:
out.shape

(4040,)

In [33]:
out[4000]

' Frina\n Fr'